In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
# Loading the api keys from your local .env file
import algo.exchange_setup

client = algo.exchange_setup.establish_connection()

In [65]:
from binance import Client
import json
import datetime
import math
from algo.datamodel import *


class MarketState:
    """ Data class that holds all the information of the market about a pair needed for our algo """

    def __init__(self, client: Client = None, window_size: int = 1000):
        assert (client != None)
        self.client = client
        self.window_size = window_size  # TODO: define smallest time unit
        self.ticker_prices = {}  # key: symbol (str), value: price (float)
        self.time = None  # TODO: implement

        self.__fetch_prices()
        self.symbols = list(self.ticker_prices.keys())

    def update(self):
        """
        Updates the MarketState by fetching the latest data from the exchange.
        This function should be called on every tick (as defined by the algorithm).
        Other functions in this class will rely on the internal states that this function updates,
        so this function should be called before any other functions in each time instance.
        """
        self.__fetch_prices()

    def __fetch_prices(self):
        """Private function that updates the prices within the MarketState class"""
        # TODO: update time
        for t in self.client.get_all_tickers():
            symbol, price = t["symbol"], float(t["price"])
            self.ticker_prices[symbol] = price

    def current_price(self, coin: str) -> float:
        """Get the current price of a specific coin"""
        return self.ticker_prices[coin]

    def portfolio_balance(self) -> float:
        """Total USD value of all coins in exchange that we are holding"""
        # need to all get_asset_balance and multiply by market price of the coin-USDT pair
        # TODO: update balance inside the update function? or separately?
        balances = [Balance(**b) for b in self.client.get_account()["balances"]]
        total_balance = 0
        excluded_coins = []
        for b in balances:
            if b.asset in ("USDT", "BUSD"):
                # stablecoin values reflect real value of USD
                total_balance += b.free + b.locked
            else:
                # need to convert this cryptocurrency to USDT to get price in USD
                ticker_usdt = b.asset + 'USDT'  # remark: assume all pairs have a USDT conversion

                if not (ticker_usdt in self.ticker_prices):
                    # coins excluded from portfolio balance (no direct conversion to USD available)
                    # TODO: find a conversion from these coins to BTC to USDT (maybe try BUSD)
                    excluded_coins.append(b.asset)
                    continue

                qty = b.free + b.locked
                total_balance += qty * self.ticker_prices[ticker_usdt]

        print(
            f"Excluded the following coins from portfolio balance calulations (could not find conversion to USD): {excluded_coins}")

        return total_balance

    def hard_stop_loss(self) -> float:
        # this will move up/down as we our portfolio increases/decreases in value,
        #   do we want to set an absolute cutoff to limit exposure or are we fine with increased exposure we our portfolio grows?
        return 0.005 * self.portfolio_balance()

    def derivative_of_spread(self) -> float:
        # return the derivative of the spread price (slope of the spread)
        pass

    def current_spread(self, coin1, coin2) -> float:
        # TODO: do we want to qty of coin1 and coin2 as an arg (based on the lin-alg formula)? do we allow fractional qty?
        return self.current_price(coin1) - self.current_price(coin2)

    def spread_moving_avg(self, coin1, coin2) -> float:
        # Get Rolling Window Price Change Statistics
        # GET /api/v3/ticker
        pass

    def spread_upper_bollinger_band(self, coin1, coin2) -> float:
        pass

    def spread_lower_bollinger_band(self, coin1, coin2) -> float:
        pass


In [66]:
# Initializer
state = MarketState(client)

In [67]:
state.update()

In [68]:
# portfolio balance
portfolio_balance = state.portfolio_balance()
print(portfolio_balance)

Excluded the following coins from portfolio balance calulations (could not find conversion to USD): ['SNT', 'GAS', 'XVG', 'AST', 'OAX', 'REQ', 'VIB', 'POWR', 'AMB', 'ADX', 'ELF', 'PIVX', 'STEEM', 'SYS', 'LOOM', 'QKC', 'PHB', 'NGN', 'RUB', 'TRY', 'ZAR', 'UAH', 'AVA', 'DAI', 'IDEX', 'WBTC', 'BAKE', 'SCRT', 'CAKE', 'VIDT', 'BRL', 'AERGO']
141297.87133600007


In [69]:
state.hard_stop_loss()

Excluded the following coins from portfolio balance calulations (could not find conversion to USD): ['SNT', 'GAS', 'XVG', 'AST', 'OAX', 'REQ', 'VIB', 'POWR', 'AMB', 'ADX', 'ELF', 'PIVX', 'STEEM', 'SYS', 'LOOM', 'QKC', 'PHB', 'NGN', 'RUB', 'TRY', 'ZAR', 'UAH', 'AVA', 'DAI', 'IDEX', 'WBTC', 'BAKE', 'SCRT', 'CAKE', 'VIDT', 'BRL', 'AERGO']


706.4893566800004

In [40]:
state.symbols

['ETHBTC',
 'LTCBTC',
 'BNBBTC',
 'BTCUSDT',
 'ETHUSDT',
 'TRXBTC',
 'XRPBTC',
 'BNBUSDT',
 'LTCUSDT',
 'LTCBNB',
 'XRPUSDT',
 'XRPBNB',
 'TRXBNB',
 'TRXUSDT',
 'BNBBUSD',
 'BTCBUSD',
 'XRPBUSD',
 'ETHBUSD',
 'LTCBUSD',
 'TRXBUSD',
 'NEOBTC',
 'QTUMETH',
 'EOSETH',
 'SNTETH',
 'BNTETH',
 'GASBTC',
 'BNBETH',
 'WTCBTC',
 'LRCBTC',
 'LRCETH',
 'QTUMBTC',
 'ZRXBTC',
 'KNCBTC',
 'FUNETH',
 'NEOETH',
 'IOTABTC',
 'IOTAETH',
 'LINKBTC',
 'LINKETH',
 'XVGETH',
 'MTLBTC',
 'EOSBTC',
 'SNTBTC',
 'ETCETH',
 'ETCBTC',
 'ZECBTC',
 'ZECETH',
 'BNTBTC',
 'ASTBTC',
 'DASHBTC',
 'DASHETH',
 'OAXBTC',
 'REQBTC',
 'VIBBTC',
 'TRXETH',
 'POWRBTC',
 'POWRETH',
 'XRPETH',
 'ENJBTC',
 'ENJETH',
 'STORJBTC',
 'KMDBTC',
 'NULSBTC',
 'XMRBTC',
 'XMRETH',
 'AMBBTC',
 'BATBTC',
 'NEOUSDT',
 'NEOBNB',
 'LSKBTC',
 'LSKETH',
 'MANABTC',
 'MANAETH',
 'ADXBTC',
 'ADXETH',
 'ADABTC',
 'ADAETH',
 'XLMBTC',
 'XLMETH',
 'XLMBNB',
 'LTCETH',
 'WAVESBTC',
 'WAVESETH',
 'ICXBTC',
 'ELFBTC',
 'ELFETH',
 'RLCBTC',
 'RLCETH',


In [ ]:
account = client.get_account()
account["balances"]

In [ ]:
from collections import ChainMap

all_tickers = client.get_all_tickers()
ticker_map = {}
for t in all_tickers:
    print(Ticker(**t))

In [ ]:
print(ticker_map)

In [ ]:
ticker_map["price"]

In [ ]:
exchange_info = client.get_exchange_info()

In [ ]:
exchange_info["symbols"]